<a href="https://colab.research.google.com/github/ashamril/Text-Analytics/blob/master/webScraping_harga_runcit_grocery_pasaran_terkini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Web scraping harga runcit pasaran terkini from Lembaga Pemasaran Pertanian Persekutuan (FAMA)**

In [1]:
from bs4 import BeautifulSoup
from datetime import datetime, timedelta, date
import requests
import calendar
import re
from collections import Counter
from wordcloud import WordCloud
import pandas as pd
import matplotlib.pyplot as plt
from collections import OrderedDict  
from pandas import DataFrame
import plotly.express as px
import numpy as np

In [2]:
# # For testing
# URL = "https://sdvi2.fama.gov.my/price/direct/price/daily_commodityRpt.asp?Pricing=A&LevelCd=03&PricingDt=2021/1/1"

# dfAllNegeri = pd.DataFrame()

# response = requests.get(URL)
# if response.status_code == 200:
#   html_text = response.text
#   soup = BeautifulSoup(html_text)
#   tables = soup.findAll('table')

#   for table in tables[1:]:
#     if "Pusat" in table.text:
#       dfNegeri = pd.read_html(str(table), header=None)[0]
#       dfAllNegeri = pd.concat([dfAllNegeri, dfNegeri])
#     else:
#       print("Harga 404!".format(URL))
# else:
#   print("Link 404! {}".format(URL))

# #dfAllNegeri = dfAllNegeri[dfAllNegeri[0].str.contains("Pusat")]
# lstAllNegeri = dfAllNegeri.values.tolist()

In [3]:
# # # For testing
# i = 0
# dfAllHarga = pd.DataFrame()
# for table in tables[1:]:
#   if "Nama Varieti" in table.text:
#     if i < len(lstAllNegeri):
#       dfHarga = pd.read_html(str(table), header=None)[0]
#       dfHarga = dfHarga[~dfHarga[0].str.contains("Varieti")]
#       dfHarga["Pusat"] = str(lstAllNegeri[i])
#       dfAllHarga = pd.concat([dfAllHarga, dfHarga])
#       i += 1

# dfAllHarga.columns=["Nama Varieti", "Gred", "Unit", "HargaTinggi", "HargaPurata", "HargaRendah", "Pusat"]
# dfAllHarga['Pusat'] = dfAllHarga['Pusat'].str.replace("\['Pusat : ", "")
# dfAllHarga['Pusat'] = dfAllHarga['Pusat'].str.replace("'\]", "")
# dfAllHarga = dfAllHarga.reset_index()
# dfAllHarga = dfAllHarga.drop('index', axis = 1) 
# dfAllHarga.head()

In [4]:
def collectHarga(year, mth, day):
  global dfAllNegeri
  global dfAllHarga
  #URL = "https://sdvi2.fama.gov.my/price/direct/price/daily_commodityRpt.asp?Pricing=A&LevelCd=03&PricingDt=2021/1/1"
  URL = f"https://sdvi2.fama.gov.my/price/direct/price/daily_commodityRpt.asp?Pricing=A&LevelCd=03&PricingDt={year}/{mth}/{day}"

  response = requests.get(URL)
  if response.status_code == 200:
    html_text = response.text
    soup = BeautifulSoup(html_text)
    tables = soup.findAll('table')

    for table in tables:
      if "TIADA MAKLUMAT" in table.text:
        tiadaHarga.append("{} {} {} {}".format(year, mth, day, URL))

    for table in tables[1:]:
      if "Pusat" in table.text:
        dfNegeri = pd.read_html(str(table), header=None)[0]
        dfAllNegeri = pd.concat([dfAllNegeri, dfNegeri])

  else:
    print("Link 404! {}".format(URL))

  lstAllNegeri = dfAllNegeri.values.tolist()
  i = 0
  
  for table in tables[1:]:
    if "Nama Varieti" in table.text:
      if i < len(lstAllNegeri):
        dfHarga = pd.read_html(str(table), header=None)[0]
        dfHarga = dfHarga[~dfHarga[0].str.contains("Varieti")]
        dfHarga["Pusat"] = str(lstAllNegeri[i])
        dfHarga["Tahun"] = int(year)
        dfHarga["Bulan"] = int(mth)
        dfHarga["Hari"] = int(day)
        dfHarga["Tarikh"] = "{}/{}/{}".format(year, mth, day)
        dfAllHarga = pd.concat([dfAllHarga, dfHarga])
        i += 1

In [6]:
dfAllNegeri = pd.DataFrame()
dfAllHarga = pd.DataFrame()
tiadaHarga = []

# Data collection start from start_date to end_date
start_date = date(2020, 12, 1)
end_date = date(2021, 2, 28)
delta = timedelta(days=1)
while start_date <= end_date:
    day = start_date.strftime("%d")
    if int(day) < 10:
      day = day[1]
    mth = start_date.strftime("%m")
    if int(mth) < 10:
      mth = mth[1]
    year = start_date.strftime("%Y")
    start_date += delta
    collectHarga(year, mth, day)

dfAllHarga.columns=["Nama Varieti", "Gred", "Unit", "HargaTinggi", "HargaPurata", "HargaRendah", "Pusat", "Tahun", "Bulan", "Hari", "Tarikh"]
dfAllHarga['Pusat'] = dfAllHarga['Pusat'].str.replace("\['Pusat : ", "")
dfAllHarga['Pusat'] = dfAllHarga['Pusat'].str.replace("'\]", "")
dfAllHarga = dfAllHarga.reset_index()
dfAllHarga = dfAllHarga.drop('index', axis = 1) 

In [22]:
x = np.array(tiadaHarga)
uniqDate = np.unique(x)
uniqDate.sort()
print(uniqDate)

['2020 12 11 https://sdvi2.fama.gov.my/price/direct/price/daily_commodityRpt.asp?Pricing=A&LevelCd=03&PricingDt=2020/12/11'
 '2020 12 13 https://sdvi2.fama.gov.my/price/direct/price/daily_commodityRpt.asp?Pricing=A&LevelCd=03&PricingDt=2020/12/13'
 '2020 12 14 https://sdvi2.fama.gov.my/price/direct/price/daily_commodityRpt.asp?Pricing=A&LevelCd=03&PricingDt=2020/12/14'
 '2020 12 16 https://sdvi2.fama.gov.my/price/direct/price/daily_commodityRpt.asp?Pricing=A&LevelCd=03&PricingDt=2020/12/16'
 '2020 12 18 https://sdvi2.fama.gov.my/price/direct/price/daily_commodityRpt.asp?Pricing=A&LevelCd=03&PricingDt=2020/12/18'
 '2020 12 2 https://sdvi2.fama.gov.my/price/direct/price/daily_commodityRpt.asp?Pricing=A&LevelCd=03&PricingDt=2020/12/2'
 '2020 12 20 https://sdvi2.fama.gov.my/price/direct/price/daily_commodityRpt.asp?Pricing=A&LevelCd=03&PricingDt=2020/12/20'
 '2020 12 21 https://sdvi2.fama.gov.my/price/direct/price/daily_commodityRpt.asp?Pricing=A&LevelCd=03&PricingDt=2020/12/21'
 '2020 12 

In [24]:
dfAllHarga.head()

,Nama Varieti,Gred,Unit,HargaTinggi,HargaPurata,HargaRendah,Pusat,Tahun,Bulan,Hari,Tarikh
0,AYAM HIDUP,F.A.Q,KILOGRAM,6.20,6.20,6.20,"JOHOR BAHRU, JOHOR",2020,12,1,2020/12/1
1,AYAM PROSES STANDARD,F.A.Q,KILOGRAM,7.50,7.50,7.50,"JOHOR BAHRU, JOHOR",2020,12,1,2020/12/1
2,BAWANG BESAR KUNING/HOLLAND,F.A.Q,KILOGRAM,6.00,5.50,5.00,"JOHOR BAHRU, JOHOR",2020,12,1,2020/12/1
3,BAWANG BESAR MERAH (IMPORT),F.A.Q,KILOGRAM,9.00,8.50,8.00,"JOHOR BAHRU, JOHOR",2020,12,1,2020/12/1
4,BAWANG MERAH ROSE,F.A.Q,KILOGRAM,12.00,11.00,10.00,"JOHOR BAHRU, JOHOR",2020,12,1,2020/12/1


In [16]:
dfAllHarga.tail()

,Nama Varieti,Gred,Unit,HargaTinggi,HargaPurata,HargaRendah,Pusat,Tahun,Bulan,Hari,Tarikh
26651,TEMBIKAI MERAH TANPA BIJI,F.A.Q,KILOGRAM,4.50,4.25,4.00,"KUALA TERENGGANU, TERENGGANU",2021,2,27,2021/2/27
26652,TERUNG PANJANG/PEPARU (TANAH RENDAH),F.A.Q,KILOGRAM,8.00,7.50,7.00,"KUALA TERENGGANU, TERENGGANU",2021,2,27,2021/2/27
26653,TIMUN HIJAU,F.A.Q,KILOGRAM,3.00,3.00,3.00,"KUALA TERENGGANU, TERENGGANU",2021,2,27,2021/2/27
26654,TOMATO (TANAH TINGGI),F.A.Q,KILOGRAM,9.00,8.50,8.00,"KUALA TERENGGANU, TERENGGANU",2021,2,27,2021/2/27
26655,UBI KENTANG HOLLAND,F.A.Q,KILOGRAM,5.00,5.00,5.00,"KUALA TERENGGANU, TERENGGANU",2021,2,27,2021/2/27


In [34]:
dfAllHarga.loc[(dfAllHarga['Nama Varieti'] == "AYAM PROSES SUPER") & (dfAllHarga['Tarikh'] == "2021/2/27")]

,Nama Varieti,Gred,Unit,HargaTinggi,HargaPurata,HargaRendah,Pusat,Tahun,Bulan,Hari,Tarikh
26172,AYAM PROSES SUPER,F.A.Q,KILOGRAM,9.50,9.00,9.00,"KOTA SETAR, KEDAH",2021,2,27,2021/2/27
26210,AYAM PROSES SUPER,F.A.Q,KILOGRAM,8.00,8.00,8.00,"KOTA BHARU, KELANTAN",2021,2,27,2021/2/27
26434,AYAM PROSES SUPER,F.A.Q,KILOGRAM,10.00,9.00,9.00,"KINTA, PERAK",2021,2,27,2021/2/27
26509,AYAM PROSES SUPER,F.A.Q,KILOGRAM,9.00,8.50,8.00,"GOMBAK, SELANGOR",2021,2,27,2021/2/27
26542,AYAM PROSES SUPER,F.A.Q,KILOGRAM,10.00,10.00,9.80,"KLANG, SELANGOR",2021,2,27,2021/2/27
26623,AYAM PROSES SUPER,F.A.Q,KILOGRAM,10.80,10.80,10.80,"KUALA TERENGGANU, TERENGGANU",2021,2,27,2021/2/27


In [37]:
dfAllHarga.loc[(dfAllHarga['Nama Varieti'] == "AYAM PROSES STANDARD") & (dfAllHarga['Pusat'] == "KUALA LUMPUR, W.P. KUALA LUMPUR")]

,Nama Varieti,Gred,Unit,HargaTinggi,HargaPurata,HargaRendah,Pusat,Tahun,Bulan,Hari,Tarikh
677,AYAM PROSES STANDARD,F.A.Q,KILOGRAM,7.50,7.50,7.50,"KUALA LUMPUR, W.P. KUALA LUMPUR",2020,12,1,2020/12/1
1450,AYAM PROSES STANDARD,F.A.Q,KILOGRAM,10.50,10.50,10.50,"KUALA LUMPUR, W.P. KUALA LUMPUR",2020,12,3,2020/12/3
2175,AYAM PROSES STANDARD,F.A.Q,KILOGRAM,10.50,10.50,10.50,"KUALA LUMPUR, W.P. KUALA LUMPUR",2020,12,5,2020/12/5
2881,AYAM PROSES STANDARD,F.A.Q,KILOGRAM,7.50,7.50,7.50,"KUALA LUMPUR, W.P. KUALA LUMPUR",2020,12,8,2020/12/8
3650,AYAM PROSES STANDARD,F.A.Q,KILOGRAM,10.50,10.50,10.50,"KUALA LUMPUR, W.P. KUALA LUMPUR",2020,12,10,2020/12/10
5057,AYAM PROSES STANDARD,F.A.Q,KILOGRAM,7.50,7.50,7.50,"KUALA LUMPUR, W.P. KUALA LUMPUR",2020,12,15,2020/12/15
7182,AYAM PROSES STANDARD,F.A.Q,KILOGRAM,9.80,9.80,9.80,"KUALA LUMPUR, W.P. KUALA LUMPUR",2020,12,22,2020/12/22
9232,AYAM PROSES STANDARD,F.A.Q,KILOGRAM,9.80,9.80,9.80,"KUALA LUMPUR, W.P. KUALA LUMPUR",2020,12,29,2020/12/29
11202,AYAM PROSES STANDARD,F.A.Q,KILOGRAM,10.50,10.50,10.50,"KUALA LUMPUR, W.P. KUALA LUMPUR",2021,1,5,2021/1/5
13272,AYAM PROSES STANDARD,F.A.Q,KILOGRAM,7.50,7.50,7.50,"KUALA LUMPUR, W.P. KUALA LUMPUR",2021,1,12,2021/1/12


In [44]:
dfAllHarga[(dfAllHarga['Nama Varieti'].str.contains("cili merah", na=False, case=False)) & (dfAllHarga["Tarikh"] == "2021/2/25")]

,Nama Varieti,Gred,Unit,HargaTinggi,HargaPurata,HargaRendah,Pusat,Tahun,Bulan,Hari,Tarikh
25487,CILI MERAH KULAI/KULAI HIBRID,F.A.Q,KILOGRAM,16.00,15.50,15.00,"JOHOR BAHRU, JOHOR",2021,2,25,2021/2/25
25524,CILI MERAH KULAI/KULAI HIBRID,F.A.Q,KILOGRAM,15.00,12.00,12.00,"KOTA SETAR, KEDAH",2021,2,25,2021/2/25
25562,CILI MERAH (CHINA),F.A.Q,KILOGRAM,10.00,10.00,10.00,"KOTA BHARU, KELANTAN",2021,2,25,2021/2/25
25563,CILI MERAH (THAILAND),F.A.Q,KILOGRAM,10.00,10.00,10.00,"KOTA BHARU, KELANTAN",2021,2,25,2021/2/25
25564,CILI MERAH KULAI/KULAI HIBRID,F.A.Q,KILOGRAM,15.00,15.00,15.00,"KOTA BHARU, KELANTAN",2021,2,25,2021/2/25
25565,CILI MERAH MINYAK,F.A.Q,KILOGRAM,12.00,12.00,12.00,"KOTA BHARU, KELANTAN",2021,2,25,2021/2/25
25603,CILI MERAH KULAI/KULAI HIBRID,F.A.Q,KILOGRAM,10.00,10.00,10.00,"MELAKA TENGAH, MELAKA",2021,2,25,2021/2/25
25604,CILI MERAH MINYAK,F.A.Q,KILOGRAM,9.00,8.00,8.00,"MELAKA TENGAH, MELAKA",2021,2,25,2021/2/25
25641,CILI MERAH KULAI/KULAI HIBRID,F.A.Q,KILOGRAM,16.00,15.50,15.00,"SEREMBAN, NEGERI SEMBILAN",2021,2,25,2021/2/25
25642,CILI MERAH MINYAK,F.A.Q,KILOGRAM,13.00,12.50,12.00,"SEREMBAN, NEGERI SEMBILAN",2021,2,25,2021/2/25


In [9]:
dfAllHarga[dfAllHarga['Pusat'].str.contains("selangor", na=False, case=False)]

,Nama Varieti,Gred,Unit,HargaTinggi,HargaPurata,HargaRendah,Pusat,Tahun,Bulan,Hari,Tarikh
347,AYAM HIDUP,F.A.Q,KILOGRAM,5.70,5.70,5.70,"GOMBAK, SELANGOR",2020,12,1,2020/12/1
348,AYAM PROSES STANDARD,F.A.Q,KILOGRAM,6.80,6.65,6.50,"GOMBAK, SELANGOR",2020,12,1,2020/12/1
349,BAWANG BESAR KUNING/HOLLAND,F.A.Q,KILOGRAM,4.00,4.00,4.00,"GOMBAK, SELANGOR",2020,12,1,2020/12/1
350,BAWANG BESAR MERAH (IMPORT),F.A.Q,KILOGRAM,8.00,7.50,6.80,"GOMBAK, SELANGOR",2020,12,1,2020/12/1
351,BAWANG MERAH ROSE,F.A.Q,KILOGRAM,15.00,14.40,13.80,"GOMBAK, SELANGOR",2020,12,1,2020/12/1
...,...,...,...,...,...,...,...,...,...,...,...
26617,TEMBIKAI MERAH TANPA BIJI,F.A.Q,KILOGRAM,3.50,3.50,3.50,"PETALING, SELANGOR",2021,2,27,2021/2/27
26618,TERUNG PANJANG/PEPARU (TANAH RENDAH),F.A.Q,KILOGRAM,12.00,12.00,12.00,"PETALING, SELANGOR",2021,2,27,2021/2/27
26619,TIMUN HIJAU,F.A.Q,KILOGRAM,3.50,3.25,3.00,"PETALING, SELANGOR",2021,2,27,2021/2/27
26620,TOMATO (TANAH TINGGI),F.A.Q,KILOGRAM,8.50,8.25,8.00,"PETALING, SELANGOR",2021,2,27,2021/2/27


In [10]:
dfAllHarga[(dfAllHarga['Pusat'].str.contains("kuala", na=False, case=False)) & (dfAllHarga["Hari"] == 28)]

,Nama Varieti,Gred,Unit,HargaTinggi,HargaPurata,HargaRendah,Pusat,Tahun,Bulan,Hari,Tarikh
18193,AYAM PROSES STANDARD,F.A.Q,KILOGRAM,9.60,9.50,9.50,"KUALA TERENGGANU, TERENGGANU",2021,1,28,2021/1/28
18194,AYAM PROSES SUPER,F.A.Q,KILOGRAM,10.00,9.80,9.80,"KUALA TERENGGANU, TERENGGANU",2021,1,28,2021/1/28
18195,BAWANG BESAR KUNING/HOLLAND,F.A.Q,KILOGRAM,7.00,6.00,6.00,"KUALA TERENGGANU, TERENGGANU",2021,1,28,2021/1/28
18196,BAWANG BESAR MERAH (IMPORT),F.A.Q,KILOGRAM,6.00,6.00,5.00,"KUALA TERENGGANU, TERENGGANU",2021,1,28,2021/1/28
18197,BAWANG MERAH ROSE,F.A.Q,KILOGRAM,23.00,20.00,20.00,"KUALA TERENGGANU, TERENGGANU",2021,1,28,2021/1/28
...,...,...,...,...,...,...,...,...,...,...,...
18395,TEMBIKAI MERAH TANPA BIJI,F.A.Q,KILOGRAM,4.50,4.25,4.00,"KUALA LUMPUR, W.P. KUALA LUMPUR",2021,1,28,2021/1/28
18396,TERUNG PANJANG/PEPARU (TANAH RENDAH),F.A.Q,KILOGRAM,6.00,6.00,6.00,"KUALA LUMPUR, W.P. KUALA LUMPUR",2021,1,28,2021/1/28
18397,TIMUN HIJAU,F.A.Q,KILOGRAM,4.00,3.50,3.00,"KUALA LUMPUR, W.P. KUALA LUMPUR",2021,1,28,2021/1/28
18398,TOMATO (TANAH TINGGI),F.A.Q,KILOGRAM,9.00,8.50,8.00,"KUALA LUMPUR, W.P. KUALA LUMPUR",2021,1,28,2021/1/28


In [11]:
dfAllHarga.loc[(dfAllHarga['Bulan'] == 2) & (dfAllHarga['Pusat'] == "SEREMBAN, NEGERI SEMBILAN")].head()

,Nama Varieti,Gred,Unit,HargaTinggi,HargaPurata,HargaRendah,Pusat,Tahun,Bulan,Hari,Tarikh
19282,AYAM PROSES STANDARD,F.A.Q,KILOGRAM,8.80,8.65,8.50,"SEREMBAN, NEGERI SEMBILAN",2021,2,2,2021/2/2
19283,BAWANG BESAR KUNING/HOLLAND,F.A.Q,KILOGRAM,4.00,3.75,3.50,"SEREMBAN, NEGERI SEMBILAN",2021,2,2,2021/2/2
19284,BAWANG BESAR MERAH (INDIA),F.A.Q,KILOGRAM,4.00,3.50,3.00,"SEREMBAN, NEGERI SEMBILAN",2021,2,2,2021/2/2
19285,BAWANG MERAH/KECIL (IMPORT),F.A.Q,KILOGRAM,7.00,6.50,6.00,"SEREMBAN, NEGERI SEMBILAN",2021,2,2,2021/2/2
19286,BAWANG MERAH/KECIL (THAI),F.A.Q,KILOGRAM,15.00,14.50,14.00,"SEREMBAN, NEGERI SEMBILAN",2021,2,2,2021/2/2


In [12]:
dfAllHarga.loc[dfAllHarga['Pusat'] == "KOTA SETAR, KEDAH"] 

,Nama Varieti,Gred,Unit,HargaTinggi,HargaPurata,HargaRendah,Pusat,Tahun,Bulan,Hari,Tarikh
37,AYAM PROSES STANDARD,F.A.Q,KILOGRAM,7.50,7.50,7.00,"KOTA SETAR, KEDAH",2020,12,1,2020/12/1
38,AYAM PROSES SUPER,F.A.Q,KILOGRAM,8.50,8.50,8.00,"KOTA SETAR, KEDAH",2020,12,1,2020/12/1
39,BAWANG BESAR KUNING/HOLLAND,F.A.Q,KILOGRAM,5.00,5.00,4.00,"KOTA SETAR, KEDAH",2020,12,1,2020/12/1
40,BAWANG BESAR MERAH (INDIA),F.A.Q,KILOGRAM,10.00,9.00,9.00,"KOTA SETAR, KEDAH",2020,12,1,2020/12/1
41,BAWANG MERAH ROSE,F.A.Q,KILOGRAM,18.00,16.00,16.00,"KOTA SETAR, KEDAH",2020,12,1,2020/12/1
...,...,...,...,...,...,...,...,...,...,...,...
26204,TEMBIKAI MERAH TANPA BIJI,F.A.Q,KILOGRAM,3.50,3.50,3.00,"KOTA SETAR, KEDAH",2021,2,27,2021/2/27
26205,TERUNG PANJANG/PEPARU (TANAH RENDAH),F.A.Q,KILOGRAM,7.00,7.00,6.00,"KOTA SETAR, KEDAH",2021,2,27,2021/2/27
26206,TIMUN HIJAU,F.A.Q,KILOGRAM,3.00,3.00,2.50,"KOTA SETAR, KEDAH",2021,2,27,2021/2/27
26207,TOMATO (TANAH TINGGI),F.A.Q,KILOGRAM,7.00,7.00,6.50,"KOTA SETAR, KEDAH",2021,2,27,2021/2/27
